# Concurrency Control: Locking & MVCC

Concurrency control is essential in database systems to ensure data consistency when multiple transactions access the same data simultaneously. This notebook explores the two primary strategies: **Locking** (pessimistic and optimistic) and **Multi-Version Concurrency Control (MVCC)**.

## 1. Pessimistic vs Optimistic Locking

### Pessimistic Locking

**Pessimistic locking** assumes conflicts are likely and prevents them by acquiring locks before accessing data.

| Aspect | Description |
|--------|-------------|
| **Philosophy** | "Conflicts will happen, so lock first" |
| **Lock Timing** | Acquired before read/write operations |
| **Lock Types** | Shared (S) for reads, Exclusive (X) for writes |
| **Use Case** | High-contention environments, financial transactions |
| **Pros** | Guarantees no conflicts, predictable behavior |
| **Cons** | Can cause deadlocks, reduces concurrency |

```sql
-- PostgreSQL example: SELECT FOR UPDATE
BEGIN;
SELECT * FROM accounts WHERE id = 1 FOR UPDATE;  -- Exclusive lock
UPDATE accounts SET balance = balance - 100 WHERE id = 1;
COMMIT;
```

### Optimistic Locking

**Optimistic locking** assumes conflicts are rare and checks for conflicts only at commit time.

| Aspect | Description |
|--------|-------------|
| **Philosophy** | "Conflicts are rare, check at the end" |
| **Lock Timing** | No locks during operation, validation at commit |
| **Mechanism** | Version numbers, timestamps, or checksums |
| **Use Case** | Low-contention environments, web applications |
| **Pros** | Higher concurrency, no deadlocks |
| **Cons** | Wasted work on conflicts, retry overhead |

```sql
-- Optimistic locking with version column
UPDATE accounts 
SET balance = balance - 100, version = version + 1
WHERE id = 1 AND version = 5;  -- Only succeeds if version matches
-- If 0 rows affected, another transaction modified the record
```

## 2. Lock Types and Compatibility

### Lock Compatibility Matrix

| Held \ Requested | Shared (S) | Exclusive (X) |
|------------------|------------|---------------|
| **Shared (S)**   | ✅ Compatible | ❌ Conflict |
| **Exclusive (X)**| ❌ Conflict   | ❌ Conflict |

### Two-Phase Locking (2PL)

The **Two-Phase Locking** protocol ensures serializability:

1. **Growing Phase**: Transaction acquires all needed locks
2. **Shrinking Phase**: Transaction releases locks (no new locks allowed)

```
Growing Phase          Lock Point          Shrinking Phase
    ┌──────────────────────┬──────────────────────┐
    │  Acquire locks       │  Release locks       │
    │  ▲ ▲ ▲               │               ▼ ▼ ▼  │
    └──────────────────────┴──────────────────────┘
```

## 3. Multi-Version Concurrency Control (MVCC)

**MVCC** maintains multiple versions of data to allow readers and writers to operate concurrently without blocking each other.

### How MVCC Works

```
┌─────────────────────────────────────────────────────────────────┐
│                        MVCC Data Structure                       │
├─────────────────────────────────────────────────────────────────┤
│  Row ID: 1                                                       │
│  ┌──────────┬────────────┬────────────┬───────────┐             │
│  │ Version  │ Created At │ Expired At │   Value   │             │
│  ├──────────┼────────────┼────────────┼───────────┤             │
│  │    1     │   TXN 100  │   TXN 200  │   $500    │ ← Old       │
│  │    2     │   TXN 200  │   TXN 350  │   $450    │ ← Expired   │
│  │    3     │   TXN 350  │     ∞      │   $600    │ ← Current   │
│  └──────────┴────────────┴────────────┴───────────┘             │
└─────────────────────────────────────────────────────────────────┘
```

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Snapshot Isolation** | Each transaction sees a consistent snapshot of the database |
| **Read Timestamp** | Determines which version a transaction can see |
| **Write Timestamp** | Records when a version was created |
| **Garbage Collection** | Removes old versions no longer needed |

### MVCC vs Locking

| Feature | MVCC | Traditional Locking |
|---------|------|--------------------|
| Readers block writers | ❌ No | ✅ Yes |
| Writers block readers | ❌ No | ✅ Yes |
| Storage overhead | Higher (multiple versions) | Lower |
| Deadlock risk | Lower | Higher |
| Popular in | PostgreSQL, MySQL InnoDB, Oracle | SQL Server (default) |

In [ ]:
import threading
import time
import random
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
from enum import Enum
from datetime import datetime
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

print("Libraries imported successfully!")

## 4. Simulating Lock Acquisition

In [ ]:
class LockType(Enum):
    SHARED = "S"      # Read lock
    EXCLUSIVE = "X"   # Write lock

@dataclass
class LockRequest:
    txn_id: str
    resource: str
    lock_type: LockType
    request_time: float
    grant_time: Optional[float] = None
    release_time: Optional[float] = None

class LockManager:
    """Simple lock manager demonstrating pessimistic locking."""
    
    def __init__(self):
        self.locks: Dict[str, List[Tuple[str, LockType]]] = {}  # resource -> [(txn_id, lock_type)]
        self.wait_queue: Dict[str, List[LockRequest]] = {}  # resource -> waiting requests
        self.history: List[LockRequest] = []
        self._lock = threading.Lock()
    
    def _is_compatible(self, existing: List[Tuple[str, LockType]], requested: LockType, txn_id: str) -> bool:
        """Check if requested lock is compatible with existing locks."""
        for holder_txn, holder_type in existing:
            if holder_txn == txn_id:
                continue  # Same transaction, allow upgrade
            if requested == LockType.EXCLUSIVE or holder_type == LockType.EXCLUSIVE:
                return False  # Exclusive locks are incompatible
        return True
    
    def acquire(self, txn_id: str, resource: str, lock_type: LockType) -> LockRequest:
        """Attempt to acquire a lock."""
        request = LockRequest(txn_id, resource, lock_type, time.time())
        
        with self._lock:
            if resource not in self.locks:
                self.locks[resource] = []
            
            existing = self.locks[resource]
            
            # Check if lock can be granted immediately
            if self._is_compatible(existing, lock_type, txn_id):
                self.locks[resource].append((txn_id, lock_type))
                request.grant_time = time.time()
                self.history.append(request)
                print(f"  ✅ {txn_id}: Acquired {lock_type.value} lock on {resource}")
                return request
            else:
                # Must wait
                print(f"  ⏳ {txn_id}: Waiting for {lock_type.value} lock on {resource}")
                if resource not in self.wait_queue:
                    self.wait_queue[resource] = []
                self.wait_queue[resource].append(request)
                self.history.append(request)
                return request
    
    def release(self, txn_id: str, resource: str):
        """Release a lock and grant to waiting transactions."""
        with self._lock:
            if resource in self.locks:
                self.locks[resource] = [(t, lt) for t, lt in self.locks[resource] if t != txn_id]
                
                # Update history
                for req in self.history:
                    if req.txn_id == txn_id and req.resource == resource and req.release_time is None:
                        req.release_time = time.time()
                        print(f"  🔓 {txn_id}: Released lock on {resource}")
                
                # Grant waiting locks
                self._process_wait_queue(resource)
    
    def _process_wait_queue(self, resource: str):
        """Process waiting requests for a resource."""
        if resource not in self.wait_queue:
            return
        
        granted = []
        for request in self.wait_queue[resource]:
            existing = self.locks.get(resource, [])
            if self._is_compatible(existing, request.lock_type, request.txn_id):
                self.locks[resource].append((request.txn_id, request.lock_type))
                request.grant_time = time.time()
                granted.append(request)
                print(f"  ✅ {request.txn_id}: Granted waiting {request.lock_type.value} lock on {resource}")
        
        for req in granted:
            self.wait_queue[resource].remove(req)

# Create lock manager instance
lock_manager = LockManager()
print("LockManager class created!")

In [ ]:
# Simulate concurrent transactions with locking
print("=" * 60)
print("Simulating Pessimistic Locking Scenario")
print("=" * 60)

lock_manager = LockManager()
start_time = time.time()

def transaction_1():
    """Transaction 1: Read and update account A."""
    print("\n[T1] Starting: Transfer from Account A")
    
    # Acquire exclusive lock for update
    lock_manager.acquire("T1", "Account_A", LockType.EXCLUSIVE)
    time.sleep(0.3)  # Simulate work
    
    # Also need to read Account B
    lock_manager.acquire("T1", "Account_B", LockType.SHARED)
    time.sleep(0.2)  # Simulate reading
    
    # Release locks
    lock_manager.release("T1", "Account_B")
    lock_manager.release("T1", "Account_A")
    print("[T1] Completed")

def transaction_2():
    """Transaction 2: Read Account A and B."""
    time.sleep(0.1)  # Start slightly after T1
    print("\n[T2] Starting: Read Accounts A and B")
    
    # Try to read Account A (will wait for T1's exclusive lock)
    lock_manager.acquire("T2", "Account_A", LockType.SHARED)
    time.sleep(0.1)
    
    lock_manager.acquire("T2", "Account_B", LockType.SHARED)
    time.sleep(0.1)
    
    lock_manager.release("T2", "Account_A")
    lock_manager.release("T2", "Account_B")
    print("[T2] Completed")

def transaction_3():
    """Transaction 3: Read Account B only."""
    time.sleep(0.15)
    print("\n[T3] Starting: Read Account B")
    
    # Can share read lock with T1
    lock_manager.acquire("T3", "Account_B", LockType.SHARED)
    time.sleep(0.15)
    
    lock_manager.release("T3", "Account_B")
    print("[T3] Completed")

# Run transactions concurrently
threads = [
    threading.Thread(target=transaction_1),
    threading.Thread(target=transaction_2),
    threading.Thread(target=transaction_3),
]

for t in threads:
    t.start()

for t in threads:
    t.join()

print("\n" + "=" * 60)
print("All transactions completed!")
print("=" * 60)

## 5. Simulating MVCC with Version Tracking

In [ ]:
@dataclass
class RowVersion:
    """Represents a single version of a row in MVCC."""
    version: int
    value: any
    created_by_txn: int
    created_at: float
    expired_by_txn: Optional[int] = None
    expired_at: Optional[float] = None

@dataclass
class Transaction:
    """Represents a transaction in MVCC."""
    txn_id: int
    start_time: float
    commit_time: Optional[float] = None
    status: str = "active"  # active, committed, aborted

class MVCCDatabase:
    """Simple MVCC implementation demonstrating version tracking."""
    
    def __init__(self):
        self.data: Dict[str, List[RowVersion]] = {}  # key -> list of versions
        self.transactions: Dict[int, Transaction] = {}
        self.next_txn_id = 1
        self.operation_log: List[dict] = []
        self._lock = threading.Lock()
    
    def begin_transaction(self) -> Transaction:
        """Start a new transaction."""
        with self._lock:
            txn = Transaction(self.next_txn_id, time.time())
            self.transactions[txn.txn_id] = txn
            self.next_txn_id += 1
            self.operation_log.append({
                'txn_id': txn.txn_id,
                'operation': 'BEGIN',
                'timestamp': txn.start_time
            })
            print(f"📝 Transaction {txn.txn_id} started")
            return txn
    
    def _get_visible_version(self, key: str, txn: Transaction) -> Optional[RowVersion]:
        """Get the version visible to a transaction (snapshot isolation)."""
        if key not in self.data:
            return None
        
        visible = None
        for version in self.data[key]:
            creator_txn = self.transactions.get(version.created_by_txn)
            
            # Version is visible if:
            # 1. Created by a committed transaction that started before our snapshot
            # 2. Or created by our own transaction
            if version.created_by_txn == txn.txn_id:
                visible = version
            elif creator_txn and creator_txn.status == "committed":
                if creator_txn.commit_time and creator_txn.commit_time < txn.start_time:
                    if version.expired_at is None or version.expired_at > txn.start_time:
                        visible = version
        
        return visible
    
    def read(self, txn: Transaction, key: str) -> Optional[any]:
        """Read a value using snapshot isolation."""
        version = self._get_visible_version(key, txn)
        value = version.value if version else None
        
        self.operation_log.append({
            'txn_id': txn.txn_id,
            'operation': 'READ',
            'key': key,
            'value': value,
            'version': version.version if version else None,
            'timestamp': time.time()
        })
        
        print(f"  👁️  T{txn.txn_id} READ {key} = {value} (version {version.version if version else 'N/A'})")
        return value
    
    def write(self, txn: Transaction, key: str, value: any):
        """Write a new version of a value."""
        with self._lock:
            if key not in self.data:
                self.data[key] = []
            
            # Expire the current visible version
            current = self._get_visible_version(key, txn)
            if current and current.expired_by_txn is None:
                current.expired_by_txn = txn.txn_id
                current.expired_at = time.time()
            
            # Create new version
            new_version = len(self.data[key]) + 1
            row = RowVersion(
                version=new_version,
                value=value,
                created_by_txn=txn.txn_id,
                created_at=time.time()
            )
            self.data[key].append(row)
            
            self.operation_log.append({
                'txn_id': txn.txn_id,
                'operation': 'WRITE',
                'key': key,
                'value': value,
                'version': new_version,
                'timestamp': time.time()
            })
            
            print(f"  ✏️  T{txn.txn_id} WRITE {key} = {value} (new version {new_version})")
    
    def commit(self, txn: Transaction):
        """Commit a transaction."""
        with self._lock:
            txn.commit_time = time.time()
            txn.status = "committed"
            
            self.operation_log.append({
                'txn_id': txn.txn_id,
                'operation': 'COMMIT',
                'timestamp': txn.commit_time
            })
            
            print(f"✅ Transaction {txn.txn_id} committed")
    
    def get_version_history(self, key: str) -> pd.DataFrame:
        """Get version history for visualization."""
        if key not in self.data:
            return pd.DataFrame()
        
        records = []
        for v in self.data[key]:
            records.append({
                'Key': key,
                'Version': v.version,
                'Value': v.value,
                'Created By': f"T{v.created_by_txn}",
                'Expired By': f"T{v.expired_by_txn}" if v.expired_by_txn else "Active",
                'Status': 'Expired' if v.expired_by_txn else 'Current'
            })
        
        return pd.DataFrame(records)

print("MVCCDatabase class created!")

In [ ]:
# Demonstrate MVCC with concurrent transactions
print("=" * 60)
print("Simulating MVCC Scenario: Bank Account Transfer")
print("=" * 60)

db = MVCCDatabase()
base_time = time.time()

# Initialize data with a committed transaction
init_txn = db.begin_transaction()
db.write(init_txn, "balance_A", 1000)
db.write(init_txn, "balance_B", 500)
db.commit(init_txn)
time.sleep(0.1)  # Ensure commit is visible

print("\n" + "-" * 40)
print("Initial state set. Starting concurrent transactions...")
print("-" * 40)

# T2: Reads account balances (will see snapshot)
t2 = db.begin_transaction()
time.sleep(0.05)

# T3: Updates balance_A (transfer out)
t3 = db.begin_transaction()

print("\n[T2 reads before T3 commits - sees old values]")
db.read(t2, "balance_A")  # T2 sees 1000 (snapshot isolation)
db.read(t2, "balance_B")  # T2 sees 500

print("\n[T3 performs transfer: A-200, B+200]")
db.write(t3, "balance_A", 800)   # 1000 - 200
db.write(t3, "balance_B", 700)   # 500 + 200
db.commit(t3)

print("\n[T2 reads again - still sees old snapshot!]")
db.read(t2, "balance_A")  # Still sees 1000 (snapshot!)
db.read(t2, "balance_B")  # Still sees 500
db.commit(t2)

# T4: New transaction sees committed changes
print("\n[T4 (new transaction) sees latest values]")
t4 = db.begin_transaction()
db.read(t4, "balance_A")  # Sees 800
db.read(t4, "balance_B")  # Sees 700
db.commit(t4)

print("\n" + "=" * 60)
print("MVCC demonstration complete!")
print("=" * 60)

In [ ]:
# Display version history
print("\n📊 Version History for 'balance_A':")
display(db.get_version_history("balance_A"))

print("\n📊 Version History for 'balance_B':")
display(db.get_version_history("balance_B"))

## 6. Plotly Timeline Visualization of Concurrent Transactions

In [ ]:
# Create timeline data for visualization
timeline_data = []

# Simulated transaction timeline (for clear visualization)
transactions_timeline = [
    {"Task": "T1 (Init)", "Start": 0, "End": 2, "Resource": "Init", "Description": "Initialize balances"},
    {"Task": "T2 (Reader)", "Start": 3, "End": 12, "Resource": "Read", "Description": "Read with snapshot"},
    {"Task": "T3 (Writer)", "Start": 4, "End": 8, "Resource": "Write", "Description": "Transfer funds"},
    {"Task": "T4 (Reader)", "Start": 13, "End": 15, "Resource": "Read", "Description": "See new values"},
]

# Operations within transactions
operations = [
    {"Task": "T1 (Init)", "Time": 0.5, "Op": "WRITE balance_A=1000"},
    {"Task": "T1 (Init)", "Time": 1, "Op": "WRITE balance_B=500"},
    {"Task": "T1 (Init)", "Time": 1.8, "Op": "COMMIT"},
    {"Task": "T2 (Reader)", "Time": 4, "Op": "READ A=1000 👁️"},
    {"Task": "T2 (Reader)", "Time": 5, "Op": "READ B=500 👁️"},
    {"Task": "T3 (Writer)", "Time": 5, "Op": "WRITE A=800"},
    {"Task": "T3 (Writer)", "Time": 6, "Op": "WRITE B=700"},
    {"Task": "T3 (Writer)", "Time": 7.5, "Op": "COMMIT"},
    {"Task": "T2 (Reader)", "Time": 9, "Op": "READ A=1000 👁️ (snapshot!)"},
    {"Task": "T2 (Reader)", "Time": 10, "Op": "READ B=500 👁️ (snapshot!)"},
    {"Task": "T2 (Reader)", "Time": 11.5, "Op": "COMMIT"},
    {"Task": "T4 (Reader)", "Time": 13.5, "Op": "READ A=800 ✨"},
    {"Task": "T4 (Reader)", "Time": 14, "Op": "READ B=700 ✨"},
    {"Task": "T4 (Reader)", "Time": 14.8, "Op": "COMMIT"},
]

df_timeline = pd.DataFrame(transactions_timeline)
df_ops = pd.DataFrame(operations)

print("Timeline data prepared!")

In [ ]:
# Create Gantt-style timeline chart
fig = go.Figure()

# Color mapping
colors = {
    "T1 (Init)": "#2ecc71",
    "T2 (Reader)": "#3498db",
    "T3 (Writer)": "#e74c3c",
    "T4 (Reader)": "#9b59b6"
}

# Add transaction bars
for idx, row in df_timeline.iterrows():
    fig.add_trace(go.Bar(
        x=[row['End'] - row['Start']],
        y=[row['Task']],
        base=[row['Start']],
        orientation='h',
        name=row['Task'],
        marker_color=colors[row['Task']],
        opacity=0.6,
        text=row['Description'],
        textposition='inside',
        showlegend=True,
        hovertemplate=f"<b>{row['Task']}</b><br>" +
                      f"Duration: {row['Start']} - {row['End']}<br>" +
                      f"{row['Description']}<extra></extra>"
    ))

# Add operation markers
for idx, row in df_ops.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['Time']],
        y=[row['Task']],
        mode='markers+text',
        marker=dict(size=12, color=colors[row['Task']], symbol='diamond'),
        text=[row['Op']],
        textposition='top center',
        textfont=dict(size=9),
        showlegend=False,
        hovertemplate=f"<b>{row['Op']}</b><br>Time: {row['Time']}<extra></extra>"
    ))

# Add vertical line showing T3 commit time
fig.add_vline(x=7.5, line_dash="dash", line_color="red", 
              annotation_text="T3 Commits", annotation_position="top")

# Layout
fig.update_layout(
    title=dict(
        text="<b>MVCC Transaction Timeline</b><br><sup>Demonstrating Snapshot Isolation</sup>",
        x=0.5
    ),
    xaxis_title="Time (relative units)",
    yaxis_title="Transaction",
    barmode='overlay',
    height=450,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    xaxis=dict(range=[-0.5, 16]),
    yaxis=dict(categoryorder='array', 
               categoryarray=['T4 (Reader)', 'T3 (Writer)', 'T2 (Reader)', 'T1 (Init)'])
)

fig

In [ ]:
# Create Lock Conflict Visualization
lock_timeline = [
    {"Task": "T1", "Resource": "Account A", "Lock": "X", "Start": 0, "End": 5, "Status": "Held"},
    {"Task": "T1", "Resource": "Account B", "Lock": "S", "Start": 3, "End": 5, "Status": "Held"},
    {"Task": "T2", "Resource": "Account A", "Lock": "S", "Start": 1, "End": 5, "Status": "Waiting"},
    {"Task": "T2", "Resource": "Account A", "Lock": "S", "Start": 5, "End": 7, "Status": "Held"},
    {"Task": "T2", "Resource": "Account B", "Lock": "S", "Start": 5.5, "End": 7, "Status": "Held"},
    {"Task": "T3", "Resource": "Account B", "Lock": "S", "Start": 2, "End": 4, "Status": "Held"},
]

df_locks = pd.DataFrame(lock_timeline)

# Create figure
fig2 = go.Figure()

# Color by status
status_colors = {"Held": "#27ae60", "Waiting": "#e74c3c"}
lock_symbols = {"S": "circle", "X": "x"}

for idx, row in df_locks.iterrows():
    color = status_colors[row['Status']]
    pattern = "" if row['Status'] == "Held" else "/"
    
    fig2.add_trace(go.Bar(
        x=[row['End'] - row['Start']],
        y=[f"{row['Task']} - {row['Resource']}"],
        base=[row['Start']],
        orientation='h',
        marker=dict(
            color=color,
            pattern_shape=pattern,
            line=dict(width=2, color='white')
        ),
        name=f"{row['Lock']} Lock ({row['Status']})",
        showlegend=False,
        text=f"{row['Lock']} ({row['Status']})",
        textposition='inside',
        hovertemplate=f"<b>{row['Task']}</b><br>" +
                      f"Resource: {row['Resource']}<br>" +
                      f"Lock: {row['Lock']} ({row['Status']})<br>" +
                      f"Time: {row['Start']} - {row['End']}<extra></extra>"
    ))

# Add legend manually
fig2.add_trace(go.Bar(x=[None], y=[None], marker_color="#27ae60", name="Lock Held", showlegend=True))
fig2.add_trace(go.Bar(x=[None], y=[None], marker_color="#e74c3c", name="Waiting", showlegend=True))

fig2.update_layout(
    title=dict(
        text="<b>Pessimistic Locking Timeline</b><br><sup>S=Shared, X=Exclusive</sup>",
        x=0.5
    ),
    xaxis_title="Time (relative units)",
    yaxis_title="Transaction - Resource",
    barmode='overlay',
    height=400,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    xaxis=dict(range=[-0.5, 8])
)

fig2.show()

## 7. Comparison Summary

In [ ]:
# Create comparison visualization
comparison_data = {
    'Aspect': ['Conflict Assumption', 'Lock Acquisition', 'Readers vs Writers', 
               'Deadlock Risk', 'Storage Overhead', 'Best For', 'Examples'],
    'Pessimistic Locking': ['Conflicts likely', 'Before access', 'Block each other', 
                            'High', 'Low', 'High contention', 'SQL Server'],
    'Optimistic Locking': ['Conflicts rare', 'At commit', 'No blocking', 
                           'None', 'Low', 'Low contention', 'Version columns'],
    'MVCC': ['Conflicts managed', 'Never for reads', 'Never block', 
             'Low', 'High (versions)', 'Mixed workloads', 'PostgreSQL']
}

df_comparison = pd.DataFrame(comparison_data)

# Create styled table
fig3 = go.Figure(data=[go.Table(
    header=dict(
        values=[f"<b>{col}</b>" for col in df_comparison.columns],
        fill_color='#2c3e50',
        font=dict(color='white', size=13),
        align='center',
        height=35
    ),
    cells=dict(
        values=[df_comparison[col] for col in df_comparison.columns],
        fill_color=[['#ecf0f1']*len(df_comparison), 
                    ['#fadbd8']*len(df_comparison),
                    ['#d5f5e3']*len(df_comparison),
                    ['#d4e6f1']*len(df_comparison)],
        font=dict(size=12),
        align='center',
        height=30
    )
)])

fig3.update_layout(
    title=dict(text="<b>Concurrency Control Comparison</b>", x=0.5),
    height=350,
    margin=dict(l=20, r=20, t=60, b=20)
)

fig3.show()

## 📌 Key Takeaways

### Locking Strategies

| Strategy | When to Use |
|----------|-------------|
| **Pessimistic** | High-contention scenarios (banking, inventory), when conflicts are costly |
| **Optimistic** | Low-contention scenarios (web apps), when retries are cheap |

### MVCC Benefits

1. **No Read-Write Blocking**: Readers never block writers and vice versa
2. **Snapshot Isolation**: Each transaction sees a consistent view of data
3. **Better Concurrency**: Higher throughput for mixed read-write workloads
4. **Trade-off**: Increased storage for version history, garbage collection needed

### Best Practices

```python
# ✅ DO: Use optimistic locking for web applications
UPDATE products SET stock = stock - 1, version = version + 1
WHERE id = ? AND version = ?  -- Check version matches

# ✅ DO: Use pessimistic locking for critical sections
SELECT * FROM accounts WHERE id = ? FOR UPDATE  -- Exclusive lock

# ✅ DO: Keep transactions short to reduce lock contention
# ✅ DO: Acquire locks in consistent order to prevent deadlocks

# ❌ DON'T: Hold locks while waiting for user input
# ❌ DON'T: Use long-running transactions with pessimistic locks
```

### Database-Specific Implementations

| Database | Default Concurrency Model |
|----------|---------------------------|
| PostgreSQL | MVCC (Snapshot Isolation) |
| MySQL InnoDB | MVCC + Row-level locking |
| Oracle | MVCC |
| SQL Server | Locking (MVCC optional with SNAPSHOT) |
| SQLite | Database-level locking |